In [1]:
from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
from sklearn.ensemble import BaggingClassifier
import lightgbm as lgb
import gc
from sklearn.pipeline import  Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import RidgeClassifierCV,LogisticRegression
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import FunctionTransformer
from copy import copy
import pickle
from sklearn.model_selection import train_test_split
import copy as cp
import gc
from sklearn.model_selection import KFold,StratifiedKFold
import xgboost as xgb

In [12]:
def xgbCV(train_x, train_y, test_x, test_y,tr_weight, te_weight):
    dtrain = xgb.DMatrix(train_x, label=train_y, weight=tr_weight)
    dtest = xgb.DMatrix(test_x, label=test_y, weight=te_weight)
    evallist = [(dtest, 'eval'), (dtrain, 'train')]
    
    xgb_c_param = {'objective': 'binary:logistic', 'max_depth': 7, 'eta': 0.1, 'silent': 1,
                            'eval_metric': 'logloss', 'max_leaf_nodes': 10}
    
    bst = xgb.train(xgb_c_param, dtrain, 100, evallist)
    
    prob = bst.predict(dtest)
    score = log_loss(test_y, prob)
    print(score)
    print(prob)
    return prob

In [13]:
def valid_feature_importance(x,y,features):
    N_SPLITS=5
    kf=KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

    predicts = pd.DataFrame()

    x['index'] = list(range(x.shape[0]))
    for i,(train_index,test_index) in enumerate(kf.split(x,y)):
        x_tr=x.iloc[train_index]
        y_tr=y.iloc[train_index]
        tr_weight = x.iloc[train_index]["weight"]
        lgb_train = lgb.Dataset(x_tr[features], y_tr, weight=tr_weight)

        x_te=x.iloc[test_index]
        y_te=y.iloc[test_index]
        te_weight = x.iloc[test_index]["weight"]
        lgb_eval = lgb.Dataset(x_te[features], y_te, weight=te_weight)

        params = {
            'num_leaves': 63,
            'objective': 'binary',
            'min_data_in_leaf':50,
            'learning_rate': 0.1,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,    
            'bagging_freq': 1,
            'max_depth':7,
            'metric': 'binary_logloss',    
        }

        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=100,
                        valid_sets=lgb_eval,
                        early_stopping_rounds=200)
        y_pred = gbm.predict(x_te[features], num_iteration=gbm.best_iteration)
        
        
        xgb_pred = xgbCV(x_tr[features], y_tr, x_te[features], y_te, tr_weight, te_weight)
        ratio = 0.8
        y_pred = ratio * y_pred + (1 - ratio) * xgb_pred
        
        tmp = pd.DataFrame({"index":x_te["index"],"predict":y_pred})
        predicts = pd.concat([tmp,predicts])
        
        del lgb_train
        del lgb_eval
        del x_tr
        del x_te
        del y_tr
        del y_te
        gc.collect()
    predicts.sort_values(by=['index'], inplace=True)
    score = log_loss(y, predicts["predict"])
    print('The rmse of prediction is:{}'.format(score))
    return score

In [4]:
pd.set_option('display.height',1000)
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

In [6]:
#读数据
gc.collect()
user_feature = pickle.load(open('../../data/processed/user_feature.pkl','rb+'))
shop_feature = pickle.load(open("../../data/processed/shop_feature.pkl","rb+"))
item_feature = pickle.load(open('../../data/processed/item_feature.pkl',"rb+"))
context_feature = pickle.load(open('../../data/processed/context_feature.pkl',"rb+"))
leak_feature = pickle.load(open("../../data/processed/leak_feature.pkl","rb+"))
current_feature = pickle.load(open("../../data/processed/current_feature.pkl","rb+"))
global_feature = pickle.load(open("../../data/processed/global_feature.pkl","rb+"))
history_feature = pickle.load(open("../../data/processed/preday_history_feature.pkl","rb+"))
history_31_6_feature = pickle.load(open("../../data/processed/history_31_6_cvr.pkl","rb+"))
diff_feature = pickle.load(open("../../data/processed/diff_feature.pkl","rb+"))
magic_feature = pickle.load(open("../../data/processed/magric_feature.pkl","rb+"))
flow_feature = pickle.load(open("../../data/processed/flow_feature.pkl","rb+"))
word2vec = pickle.load(open('../../data/processed/all_data_with_history7_1_tw2v.pkl','rb+'))



#user_feature.pop("is_trade")
shop_feature.pop("is_trade")
item_feature.pop("is_trade")
context_feature.pop("is_trade")
leak_feature.pop("is_trade")
current_feature.pop("is_trade")
global_feature.pop("is_trade")
history_feature.pop("is_trade")
history_31_6_feature.pop("is_trade")
diff_feature.pop("is_trade")
magic_feature.pop("is_trade")
flow_feature.pop("is_trade")




user_feature = user_feature.reset_index(drop=True)
shop_feature = shop_feature.reset_index(drop=True)
item_feature = item_feature.reset_index(drop=True)
context_feature = context_feature.reset_index(drop=True)
leak_feature = leak_feature.reset_index(drop=True)
current_feature = current_feature.reset_index(drop=True)
global_feature = global_feature.reset_index(drop=True)
history_feature = history_feature.reset_index(drop=True)
history_31_6_feature = history_31_6_feature.reset_index(drop=True)
diff_feature = diff_feature.reset_index(drop=True)
magic_feature = magic_feature.reset_index(drop=True)
flow_feature = flow_feature.reset_index(drop=True)
word2vec = word2vec.reset_index(drop=True)

del diff_feature["shop_id"]
#
del word2vec["is_trade"]
del word2vec["user_id"]
del word2vec["item_id"]

df = pd.concat([user_feature,
               shop_feature,
               item_feature,
               context_feature,
               leak_feature,
               current_feature,
               global_feature,
               history_feature,
               history_31_6_feature,
               diff_feature,
               magic_feature,
               flow_feature,
               word2vec,
 
               ],axis=1)


In [7]:
#特征集合
base_features = ['user_age_level', 'user_occupation_id', 'user_star_level', 'user_gender_id', 'shop_review_num_level', 
                 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description',
                 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'negative_seconds_gap',
                 'negative_visited_item', 'negative_visited_shop', 'is_last_today', 'count_global_user_id_item_id', 'positive_visited_item',
                 'count_global_user_id_shop_id', 'positive_is_same_item_id', 'cvr_31_5_item_id', 'positive_visited_shop', 'cvr_31_5_shop_id',
                 'positive_is_same_shop_id', 'positive_seconds_gap', 'trades_31_5_item_id', 'ctr_item_id', 'ctr_shop_id',
                 'trades_31_5_shop_id', 'ctr_item_brand_id', 'item_pv_level_diff', 'property_num',
                 'count_global_user_id_item_price_level', 'count_item_id', 'counts_31_5_user_id',
                 'count_global_user_id', 'predict_category_item_category_ratio', 'count_global_user_id_shop_star_level', 
                 'count_global_item_id', 'count_user_id_shop_id', 'count_global_user_id_item_collected_level',
                 'trades_31_5_item_brand_id', 'count_item_brand_id', 'match_proetry_num', 'item_city_id',
                 'item_sales_level_diff', 'category_num', 'item_category_list1', 'item_brand_id', 'cvr_31_5_user_id',
                 'counts_31_5_user_id_item_id', 'context_page_id', 'count_user_id', 'count_shop_id', 'counts_31_5_shop_id',
                 'item_category_list2', 'item_collected_level_diff', 'average_user_id_item_id_visited_time_gap', 'ctr_user_id_shop_id',
                "apperance_nums_user_id_item_id_ratio","apperance_nums_user_id_ratio",
                 'itemid_0', 'itemid_1', 'itemid_2', 'itemid_3', 'itemid_4', 'itemid_5', 'itemid_6', 'itemid_7', 'itemid_8', 'itemid_9', 'itemid_10', 
                 'itemid_11', 'itemid_12', 'itemid_13', 'itemid_14', 'itemid_15', 'itemid_16', 'itemid_17', 'itemid_18', 'itemid_19',          
                ]

In [14]:
#构造线下与线上数据集
train = df.loc[~np.isnan(df["is_trade"])]
test = df.loc[np.isnan(df["is_trade"])]

train["weight"] = 1
#类别平衡
#train.loc[(train["is_trade"]==0), 'weight'] = 1.2349342004794424
train.loc[(train["is_trade"]==0), 'weight'] = 1.3028833033940876
y = train['is_trade']
x = train
test = test

gc.collect()

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


17

In [15]:
valid_feature_importance(x,y,base_features)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


[1]	valid_0's binary_logloss: 0.610933
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.543658
[3]	valid_0's binary_logloss: 0.487749
[4]	valid_0's binary_logloss: 0.4407
[5]	valid_0's binary_logloss: 0.400789
[6]	valid_0's binary_logloss: 0.3667
[7]	valid_0's binary_logloss: 0.337406
[8]	valid_0's binary_logloss: 0.312135
[9]	valid_0's binary_logloss: 0.29022
[10]	valid_0's binary_logloss: 0.271194
[11]	valid_0's binary_logloss: 0.254652
[12]	valid_0's binary_logloss: 0.240233
[13]	valid_0's binary_logloss: 0.227649
[14]	valid_0's binary_logloss: 0.216657
[15]	valid_0's binary_logloss: 0.207048
[16]	valid_0's binary_logloss: 0.198655
[17]	valid_0's binary_logloss: 0.191317
[18]	valid_0's binary_logloss: 0.184881
[19]	valid_0's binary_logloss: 0.179263
[20]	valid_0's binary_logloss: 0.174342
[21]	valid_0's binary_logloss: 0.170053
[22]	valid_0's binary_logloss: 0.166327
[23]	valid_0's binary_logloss: 0.163052
[24]	valid_0's binary_logloss: 

[84]	eval-logloss:0.139807	train-logloss:0.13596
[85]	eval-logloss:0.139787	train-logloss:0.135894
[86]	eval-logloss:0.139769	train-logloss:0.135809
[87]	eval-logloss:0.139766	train-logloss:0.135722
[88]	eval-logloss:0.139745	train-logloss:0.135604
[89]	eval-logloss:0.139733	train-logloss:0.135507
[90]	eval-logloss:0.139708	train-logloss:0.135428
[91]	eval-logloss:0.139687	train-logloss:0.135328
[92]	eval-logloss:0.139669	train-logloss:0.135229
[93]	eval-logloss:0.139659	train-logloss:0.135133
[94]	eval-logloss:0.139648	train-logloss:0.135046
[95]	eval-logloss:0.139635	train-logloss:0.134969
[96]	eval-logloss:0.139628	train-logloss:0.1349
[97]	eval-logloss:0.139626	train-logloss:0.1348
[98]	eval-logloss:0.139618	train-logloss:0.134714
[99]	eval-logloss:0.139614	train-logloss:0.134607
0.16960259998207158
[0.03002634 0.02809444 0.0246606  ... 0.026954   0.00353445 0.02791028]
[1]	valid_0's binary_logloss: 0.610989
Training until validation scores don't improve for 200 rounds.
[2]	valid_0

[67]	eval-logloss:0.141681	train-logloss:0.137421
[68]	eval-logloss:0.141634	train-logloss:0.137284
[69]	eval-logloss:0.141604	train-logloss:0.137165
[70]	eval-logloss:0.141564	train-logloss:0.137044
[71]	eval-logloss:0.141542	train-logloss:0.136934
[72]	eval-logloss:0.141518	train-logloss:0.136809
[73]	eval-logloss:0.141499	train-logloss:0.136727
[74]	eval-logloss:0.141474	train-logloss:0.136619
[75]	eval-logloss:0.141459	train-logloss:0.136508
[76]	eval-logloss:0.141438	train-logloss:0.136376
[77]	eval-logloss:0.141414	train-logloss:0.136288
[78]	eval-logloss:0.141384	train-logloss:0.136176
[79]	eval-logloss:0.141367	train-logloss:0.136069
[80]	eval-logloss:0.141347	train-logloss:0.135989
[81]	eval-logloss:0.141324	train-logloss:0.13589
[82]	eval-logloss:0.141309	train-logloss:0.135825
[83]	eval-logloss:0.141273	train-logloss:0.135717
[84]	eval-logloss:0.141256	train-logloss:0.135613
[85]	eval-logloss:0.141238	train-logloss:0.13552
[86]	eval-logloss:0.141217	train-logloss:0.135412
[8

[49]	eval-logloss:0.142784	train-logloss:0.139896
[50]	eval-logloss:0.142693	train-logloss:0.139736
[51]	eval-logloss:0.142605	train-logloss:0.139549
[52]	eval-logloss:0.142513	train-logloss:0.139349
[53]	eval-logloss:0.142428	train-logloss:0.13919
[54]	eval-logloss:0.142361	train-logloss:0.139029
[55]	eval-logloss:0.142289	train-logloss:0.138892
[56]	eval-logloss:0.142233	train-logloss:0.138749
[57]	eval-logloss:0.142165	train-logloss:0.138615
[58]	eval-logloss:0.142111	train-logloss:0.138466
[59]	eval-logloss:0.142058	train-logloss:0.138337
[60]	eval-logloss:0.142003	train-logloss:0.138205
[61]	eval-logloss:0.141964	train-logloss:0.138071
[62]	eval-logloss:0.141917	train-logloss:0.137952
[63]	eval-logloss:0.141876	train-logloss:0.137839
[64]	eval-logloss:0.141831	train-logloss:0.137706
[65]	eval-logloss:0.14179	train-logloss:0.137577
[66]	eval-logloss:0.141765	train-logloss:0.137436
[67]	eval-logloss:0.141736	train-logloss:0.137328
[68]	eval-logloss:0.141705	train-logloss:0.137192
[6

[31]	eval-logloss:0.148619	train-logloss:0.147123
[32]	eval-logloss:0.147834	train-logloss:0.146254
[33]	eval-logloss:0.147151	train-logloss:0.145498
[34]	eval-logloss:0.146547	train-logloss:0.144832
[35]	eval-logloss:0.146015	train-logloss:0.14422
[36]	eval-logloss:0.145566	train-logloss:0.143689
[37]	eval-logloss:0.145166	train-logloss:0.143216
[38]	eval-logloss:0.14481	train-logloss:0.142788
[39]	eval-logloss:0.144499	train-logloss:0.142407
[40]	eval-logloss:0.144225	train-logloss:0.142068
[41]	eval-logloss:0.143995	train-logloss:0.141749
[42]	eval-logloss:0.143781	train-logloss:0.141434
[43]	eval-logloss:0.1436	train-logloss:0.141177
[44]	eval-logloss:0.14343	train-logloss:0.140918
[45]	eval-logloss:0.143284	train-logloss:0.14069
[46]	eval-logloss:0.143149	train-logloss:0.140492
[47]	eval-logloss:0.143035	train-logloss:0.140287
[48]	eval-logloss:0.142891	train-logloss:0.14006
[49]	eval-logloss:0.142804	train-logloss:0.139896
[50]	eval-logloss:0.142724	train-logloss:0.139706
[51]	ev

[13]	eval-logloss:0.218135	train-logloss:0.216589
[14]	eval-logloss:0.208598	train-logloss:0.206959
[15]	eval-logloss:0.200269	train-logloss:0.198533
[16]	eval-logloss:0.192995	train-logloss:0.19117
[17]	eval-logloss:0.186639	train-logloss:0.184719
[18]	eval-logloss:0.181098	train-logloss:0.179084
[19]	eval-logloss:0.176258	train-logloss:0.174149
[20]	eval-logloss:0.172034	train-logloss:0.169832
[21]	eval-logloss:0.168344	train-logloss:0.166062
[22]	eval-logloss:0.165136	train-logloss:0.162764
[23]	eval-logloss:0.162348	train-logloss:0.159886
[24]	eval-logloss:0.159916	train-logloss:0.157368
[25]	eval-logloss:0.157808	train-logloss:0.155182
[26]	eval-logloss:0.155993	train-logloss:0.153265
[27]	eval-logloss:0.1544	train-logloss:0.151592
[28]	eval-logloss:0.153028	train-logloss:0.150125
[29]	eval-logloss:0.151844	train-logloss:0.148846
[30]	eval-logloss:0.150811	train-logloss:0.147731
[31]	eval-logloss:0.149913	train-logloss:0.146731
[32]	eval-logloss:0.149129	train-logloss:0.145862
[33

0.17152431671939963